# 두바이유 크롤링
https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_DU&fdtc=2

In [ ]:
import time
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", message="You provided Unicode markup but also provided a value for from_encoding.")
def get_oil_price(code):
    delay = 0.01
    page = 1
    result = []
    start_date = '2019.01.01'
    end_date = '2025.03.21'
    start_time = datetime.now()
    
    # 수집
    print('[{}] 데이터 수집을 시작합니다. (code: {})'.format(start_time.strftime('%Y/%m/%d %H:%M:%S'), code))
    while(True):
        url = 'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd={}&fdtc=2&page={}'.format(code, page)
        data = pd.read_html(url, encoding='euc-kr')[0].dropna()
        if page != 1:
            try:
                if start_date in data.iloc[:,0].values:
                    data = data.loc[:data.iloc[:,0].values.tolist().index(start_date),]
                    result.append(data)
                    break
            except:
                break
        result.append(data)
        page += 1
        time.sleep(delay)
    
    # 가공
    oil_price = pd.concat(result).reset_index(drop=True)
    oil_price.columns = ['date', 'price', '전일대비', '등락율']
    oil_price = oil_price.loc[oil_price['date'].tolist().index(end_date):,]
    oil_price['date'] = oil_price['date'].apply(lambda x: datetime.strptime(x, '%Y.%m.%d'))
    oil_price = oil_price[['date', 'price']]
    oil_price.insert(0, 'code', code)
    oil_price = oil_price[::-1].reset_index(drop=True)
    
    end_time = datetime.now()
    print('[{}] 데이터 수집을 종료합니다. (code: {}, 수집시간: {}초, 데이터수: {:,}개)'.format(end_time.strftime('%Y/%m/%d %H:%M:%S'), code, (end_time-start_time).seconds, len(oil_price)))
    return oil_price
    
oil_price_du = get_oil_price('OIL_DU')

[2025/03/23 20:53:34] 데이터 수집을 시작합니다. (code: OIL_DU)


# 기상청 데이터 수집(종관 기상청 데이터)

In [12]:
import requests
import pandas as pd

def select_data(city_num, start, end):
    url = "http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList"
    final_df = pd.DataFrame()

    for i in city_num:
        params = {
        "serviceKey": "iCNxo2r0TdZnnV63/ItO+QrOUqJakXCxx/m20BsCp53DGZzJMDd1/7jOGLYQE+Sn+1EQeSeIhUsTIyQ5dYgy4Q==",
        "pageNo": "1",
        "numOfRows": "999",
        "dataType": "JSON",
        "dataCd": "ASOS",
        "dateCd": "HR",
        "startDt": start,
        "startHh": "00",
        "endDt": end,
        "endHh": "23",
        "stnIds": i
    }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            print(f"데이터를 성공적으로 가져왔습니다. (stnId: {i})")

        # 실제 데이터 추출
            items = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])
            df = pd.DataFrame(items)  # DataFrame으로 변환
            final_df = pd.concat([final_df, df], ignore_index=True)  # 누적하여 데이터프레임에 추가
        else:
            print(f"요청 실패: 상태 코드 {response.status_code}, {response.text}")

    return final_df

def merge_data(a,b,c,d,e,f):
  df = pd.concat([a,b,c,d,e,f],axis=0)

  return df

,code,date,price
0,OIL_DU,2019-02-01,62.62
1,OIL_DU,2019-02-04,62.55
2,OIL_DU,2019-02-05,62.08
3,OIL_DU,2019-02-06,62.71
4,OIL_DU,2019-02-07,61.79
...,...,...,...
1338,OIL_DU,2024-05-24,83.83
1339,OIL_DU,2024-05-28,84.40
1340,OIL_DU,2024-05-29,83.49
1341,OIL_DU,2024-05-30,82.12


# 일별 연료비 추정 api 호출

In [ ]:
import requests
import pprint
import json

url = "https://apis.data.go.kr/B552115/SmpDecByFuel2/getSmpDecByFuel2"
params = {
    'serviceKey': 'H22OmOJexqCM+zsw7uefTycvxU0xHGpf1hpBO2WF2TEPma8bcdd8GP8xVkBzGnmMXE9s4aXjUgHbVUcZGkR8vQ==',
    'pageNo': 1,
    'numOfRows': 60000,
    'dataType': 'json'
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print("데이터를 성공적으로 가져왔습니다.")
    
    # 실제 데이터 추출 (구조에 따라 경로 수정 필요)
    items = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])
    
    # DataFrame으로 변환
    df = pd.DataFrame(items)
    
    # 데이터 타입 변환 및 결측치 처리
    if 'cnt' in df.columns:
        df['cnt'] = df['cnt'].astype(int)
    df = df.dropna()

In [ ]:
# 포인트 체크 함수 정의
def check_point(data1, df_selection1):  # 날짜 기준으로 포인트 확인
    # 'point_power' 열이 없으면 추가
    if "point_power" not in data1.columns:
        data1["point_power"] = None  # 기본값은 None으로 설정

    adas = pd.DataFrame()  # 빈 데이터프레임 초기화

    # 각 날짜에 대해 반복 수행
    for i in df_selection1["tradeDay"].unique():
        # i에 해당하는 날짜의 데이터 선택
        date_select = df_selection1[df_selection1["tradeDay"] == i]

        # data1에 해당 날짜가 있는 경우만 처리
        if not data1[data1["날짜"] == i].empty:
            # 'cnt'가 24인 경우 "LNG" 할당
            if not date_select[date_select["cnt"] == 24].empty:
                data1.loc[data1["날짜"] == i, "point_power"] = "LNG"
            else:
                # LNG가 아닌 연료 중 유연탄, 무연탄, 유류, 원자력 각각의 최소값 선택
                anthracite_rows = date_select[date_select["fuelType"] == "무연탄"]
                bituminous_rows = date_select[date_select["fuelType"] == "유연탄"]
                oil_rows = date_select[date_select["fuelType"] == "중유"]
                nuclear_rows = date_select[date_select["fuelType"] == "원자력"]

                # 'adas'에 'not_lng_rows' 데이터 누적
                not_lng_rows = pd.concat([anthracite_rows, bituminous_rows, oil_rows, nuclear_rows])
                adas = pd.concat([adas, not_lng_rows], ignore_index=True)

                selected_indices = set()  # 선택된 연료의 인덱스를 저장할 집합

                # 원자력 최소값 선택 (가장 우선적으로 할당)
                if not nuclear_rows.empty:
                    nuclear_min_row = nuclear_rows.loc[nuclear_rows["cnt"].idxmin()]
                    selected_indices.add(nuclear_min_row.name)
                    # 해당 날짜의 point_power에 원자력 할당
                    data1.loc[(data1["날짜"] == i) & (data1.index == nuclear_min_row.name), "point_power"] = "원자력"

                # 무연탄 최소값 선택 (이미 선택된 인덱스를 제외하고 선택)
                if not anthracite_rows.empty:
                    valid_selected_indices = [idx for idx in selected_indices if idx in anthracite_rows.index]
                    anthracite_rows_filtered = anthracite_rows.drop(index=valid_selected_indices)  # 유효한 인덱스만 제외
                    if not anthracite_rows_filtered.empty:
                        anthracite_min_row = anthracite_rows_filtered.loc[anthracite_rows_filtered["cnt"].idxmin()]
                        selected_indices.add(anthracite_min_row.name)
                        # 해당 날짜의 point_power에 무연탄 할당
                        data1.loc[(data1["날짜"] == i) & (data1.index == anthracite_min_row.name), "point_power"] = "무연탄"

                # 유연탄 최소값 선택 (이미 선택된 인덱스를 제외하고 선택)
                if not bituminous_rows.empty:
                    valid_selected_indices = [idx for idx in selected_indices if idx in bituminous_rows.index]
                    bituminous_rows_filtered = bituminous_rows.drop(index=valid_selected_indices)  # 유효한 인덱스만 제외
                    if not bituminous_rows_filtered.empty:
                        bituminous_min_row = bituminous_rows_filtered.loc[bituminous_rows_filtered["cnt"].idxmin()]
                        selected_indices.add(bituminous_min_row.name)
                        # 해당 날짜의 point_power에 유연탄 할당
                        data1.loc[(data1["날짜"] == i) & (data1.index == bituminous_min_row.name), "point_power"] = "유연탄"

                # 유류 최대값 선택 (이미 선택된 인덱스를 제외하고 선택)
                if not oil_rows.empty:
                    valid_selected_indices = [idx for idx in selected_indices if idx in oil_rows.index]
                    oil_rows_filtered = oil_rows.drop(index=valid_selected_indices)  # 유효한 인덱스만 제외
                    if not oil_rows_filtered.empty:
                        oil_max_row = oil_rows_filtered.loc[oil_rows_filtered["cnt"].idxmax()]
                        # 해당 날짜의 point_power에 유류 할당
                        data1.loc[(data1["날짜"] == i) & (data1.index == oil_max_row.name), "point_power"] = "중유"

    return adas, data1